# Deploy pyannoteAI Diarization Model Package from AWS Marketplace 

The Speaker Diarization API enables accurate segmentation of audio recordings by detecting and labeling individual speakers across time. Designed for seamless integration into transcription pipelines, media workflows, and audio analytics systems, it supports a wide range of formats including WAV, MP3, FLAC, and OGG. The service is language-agnostic and works across diverse audio sourcecalls, meetings, interviews, podcasts, and more. With built-in support for mono and stereo channels, varying sample rates, and flexible input options it can be deployed in batch or near-real-time use cases. Key features include automatic speaker count estimation, precise time-stamped speaker labeling, and detection of overlapping speech. Outputs are returned in structured JSON for easy integration with transcription engines, search indexes, or business intelligence tools. Whether you are enriching speech-to-text transcripts, analyzing call center performance, or processing long-form media, this API improves clarity, organization, and data usability.

This sample notebook shows you how to deploy the **pyannoteAI Diarization Model** using Amazon SageMaker.

> **Note**: This reference notebook cannot run unless you make the suggested changes in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements that render correctly in the Jupyter interface. Open it from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that the IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions, and you have the authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. your AWS account has a **pyannoteAI Diarization Model** subscription. If so, skip the step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create a model and perform batch inference](#2-perform-batch-inference)
    1. [Create a model](#a-create-a-model)
    2. [Run Batch Transform Job](#b-run-batch-transform-job)
3. [Cleanup](#3.-Cleanup)

We recommend using **ml.g4dn.xlarge** instance.

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page **pyannoteAI Diarization Model**.
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agree with EULA, pricing, and support terms. 
1. Once you click on the **Continue to configuration** button and choose a **region**, you will see a **Product ARN** displayed. This is the model package ARN that you need to specify while creating a deployable model using `boto3`. Copy the ARN corresponding to your region and specify it in the following cell.

In [ ]:
#This is sample ARN please replace with the subscribed ARN.
model_package_arn = "arn:aws:sagemaker:{zone}:{account_id}:model-package/xxx"

In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role
import time
import boto3

role = get_execution_role()

sagemaker_session = sage.Session()
runtime = boto3.client("runtime.sagemaker")

## 2. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
bucket_name = 'sample-s3-bucket-for-diarization-test'

# Set the local path of the folder where sample audio files are kept
sample_audio_input = '../example-input.json'

# Set the name of your S3 bucket and the name of the folder you want to create
input_folder_name = 'input'

# Create an S3 client
s3 = boto3.client('s3')

# Create the S3 bucket
s3.create_bucket(Bucket=bucket_name)

# Create the folder in the S3 bucket
s3.put_object(Bucket=bucket_name, Key=input_folder_name + '/')

# Upload file to the folder in the S3 bucket
s3.upload_file(sample_audio_input, bucket_name, input_folder_name + '/example-input.json')

print("Transform input uploaded..")
#Display the content of S3 Bucket
# List the objects in the bucket
response = s3.list_objects_v2(
    Bucket=bucket_name
)

# Print the response
for obj in response['Contents']:
    print(obj['Key'])

### A. Create a model

In [ ]:
ModelName = "pyannoteai-diarization-batch-inference-model"
sm_boto3 = boto3.client("sagemaker")
cm_res_end = sm_boto3.create_model(
    ModelName=ModelName,  # name the of the model does not need to be the same as the image repo
    Containers=[
        {            
            'ModelPackageName': model_package_arn
        },
    ],    
    ExecutionRoleArn=role,
    EnableNetworkIsolation=True,
)
print("Model Arn: " + cm_res_end['ModelArn'])

### B. Run Batch Transform Job

In [ ]:
# Run the batch-transform job
transform_job_name = "pyannoteai-diarization-batch-inference" + time.strftime("-%Y%m%d%H%M", time.gmtime())

response_create_job = sm_boto3.create_transform_job(
    TransformJobName=transform_job_name,
    ModelName=ModelName,
    MaxConcurrentTransforms=1,
    #InstanceCount=1
    MaxPayloadInMB=50,
    TransformInput={
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': "s3://"+bucket_name+"/input/"    # Input Folder
            }
        },
        'ContentType': "application/json"
    },
    TransformOutput={
        'S3OutputPath': "s3://"+bucket_name+"/output/", #output folder
        'Accept': 'application/json',
        'AssembleWith': 'Line',        
    },
    TransformResources={
        'InstanceType': 'ml.g4dn.xlarge',
        'InstanceCount': 1      
    }, 
)
print(response_create_job)

In [ ]:
print("Transform output")
#Display the content of S3 Bucket
# List the objects in the bucket
response = s3.list_objects_v2(
    Bucket=bucket_name
)

# Print the response
for obj in response['Contents']:
    print(obj['Key'])

## 3. Clean-up

### A. Delete the model

In [ ]:
sm_boto3.delete_model(ModelName=ModelName)